# 3. Maps and Graph Coloring
This jupyter notebook explores the problem of graph coloring, which will help us to define the frequencies to be used for communication in eahc zone, ensuring that no two zones sharing a boundary have the same frequency for communication. This corresponds to the coloring problem: given a graph, is it posible to color each node in a color such that no two neighboring nodes share the same color. What is the minimum number of colors required to achieve this? This is a NP-difficult problem.

We reutilize the Voronoi diagram functions that we worked on in notebook 2. We will import the same functions from `voronoi.py`. This script also contains two voronoi diagrams pre-defined: `voronoi_diagram` and `voronoi_diagram_complete`.

In [1]:
from voronoi import *
import tkinter as tk

In [2]:
window = tk.Tk()
canvas = tk.Canvas(window, width=700, height=700)
draw_cartesian(canvas)
draw_voronoi(canvas, voronoi_diagram)
canvas.pack()
window.mainloop()

The partial and complete voronoi diagrams are shown below:
![Partial and Complete Voronoi Diagram](images/partial_complete_voronoi.jpg)

Let's first try and color the voronoi diagram with random colors.

In [3]:
def random_coloring(n, voronoi):
    color = []
    for i in range(0,len(voronoi)):
        color.append(random.randint(0,n))
    return color

def int2col(n):
    colors = ["red", "blue", "green", "purple", "yellow", "lightblue", "brown", "cyan", "magenta", "orange", "lightgreen", "gray"]
    return colors[n]

def draw_polygon_colored(canvas, polygon, color):
    py_points = []
    while len(polygon) != 0:
        p = canvas_point(polygon.pop(0))
        py_points.append(p)
    canvas.create_polygon(py_points, fill=int2col(color))
    for i in range(len(py_points)):
        p=i%len(py_points)
        pnext=(i+1)%len(py_points)
        canvas.create_line(py_points[p],py_points[pnext],fill="black")

def draw_voronoi_colored(canvas, diagram, colors):
    n=0
    for i in diagram:
        if len(i[1])>=3:
            draw_polygon_colored(canvas,tri_polygone(i[1]),colors[n])
        canvas.create_text(canvas_point(i[0]),text='p'+str(n))
        n+=1

In [4]:
window = tk.Tk()
canvas = tk.Canvas(window, width=700, height=700)

draw_cartesian(canvas)

canvas.pack()
colors = random_coloring(5, voronoi_diagram_complete)
draw_voronoi_colored(canvas, voronoi_diagram_complete, colors)
window.mainloop()

![Random coloring of voronoi diagram](images/random_color.jpg)

We now try to find the neighboring cells of every cell in a voronoi diagram. The function `voronoi_neighboring()` below returns a list of neighbors where `neighbors[i]` are the indices of the `i`th cell's neighbors.

In [5]:
def voronoi_neighboring(voronoi):
  adjList = [[] for i in range(0, len(voronoi))]
  n=0
  for cell in voronoi:
      m=0
      for otherCell in voronoi:
          c=0
          if cell!=otherCell:
              for point1 in cell[1]:
                  for point2 in otherCell[1]:
                      if same_point(point1,point2) and c==0:
                          adjList[n].append(m)
                          c+=1
          m+=1
      n+=1
  return(adjList)

In [6]:
out = voronoi_neighboring(voronoi_diagram)
print(out)

[[12, 13, 19], [], [], [], [], [], [], [], [], [15, 17], [], [], [0, 13, 15, 17], [0, 12, 19], [], [9, 12, 17], [], [9, 12, 15], [], [0, 13]]


## Greedy coloring approach
First, we try a greedy coloring approach, where we first select one color, and then continue to color as many cells as we can using that color without violating the requirements. Once we have colored all the cells we could, we choose a new color and continue in a similar manner.

In [7]:
def greedy_coloring(adjlist):
    colors=[ 0 for i in range(len(adjlist))]
    colors[0]=1
    i=1
    while i < len(adjlist):
        if len(adjlist[i])>0: 
            neighcolor=[]
            for j in range(len(adjlist[i])):
                neighcolor.append(colors[adjlist[i][j]])
            for k in range(1,9):
                if k not in neighcolor:
                    colors[i] = k
                    break
        i += 1
    return colors

In [8]:
window = tk.Tk()
canvas = tk.Canvas(window, width=700, height=700)

draw_cartesian(canvas)

canvas.pack()
adj = voronoi_neighboring(voronoi_diagram_complete)
colors = greedy_coloring(adj)
draw_voronoi_colored(canvas, voronoi_diagram_complete, colors)
window.mainloop()

![Result of greedy coloring on partial and complete voronoi diagrams](images\partial_complete_greedy.jpg)

We see that for the greedy approach, 5 colors are required to fill all the cells in the `voronoi_diagram_complete`, and 4 colors are required for the `voronoi_diagram`.

This greedy appraoch can be imporved upon by simply sorting the order in which the greedy algorithm visits nodes. A **perfectly orderable graph** ([Wiki](https://en.wikipedia.org/wiki/Perfectly_orderable_graph)) is one whose vertices can be ordered in a manner such that a greedy algorithm following such an order would optimally color every *induced subgraph*. Since we do not know this 'perfect ordering', we can try some simple approached. The function `sort_decreasing_degree()` below sorts the nodes in decreasing order of the degree of its vertices.

In [9]:
def sort_decreasing_degree(graph):
    listToSort = []
    i=0
    for cell in graph:
        listToSort.append((len(cell[1]),i))
        i+=1
    L=sorted(listToSort, reverse=1)
    sortedlist=[]
    for i in range(len(L)):
        sortedlist.append(L[i][1])
    return(sortedlist)

def sort_greedy_coloring(adjlist,lst):
    colors=[ 0 for i in range(len(adjlist))]
    colors[lst[0]]=1
    i=1
    while i < len(adjlist):
        if len(adjlist[lst[i]])>0: 
            neighcolor=[]
            for j in range(len(adjlist[lst[i]])):               # colors of all the neigbours of i
                neighcolor.append(colors[adjlist[lst[i]][j]])
            for k in range(1,9):
                if k not in neighcolor:
                    colors[lst[i]] = k
                    break
        i += 1
    return colors

In [10]:
window = tk.Tk()
canvas = tk.Canvas(window, width=700, height=700)

draw_cartesian(canvas)

canvas.pack()
adj = voronoi_neighboring(voronoi_diagram_complete)
lst = sort_decreasing_degree(voronoi_diagram_complete)
colors =sort_greedy_coloring(adj,lst)
draw_voronoi_colored(canvas, voronoi_diagram_complete, colors)
window.mainloop()

![REsults of applying sorted greedy coloring on both voronoi diagrams](images/par_com_sortedGreedy.jpg)
We see that the `voronoi_diagram_complete` still requires 5 colors to be completely filled. However, for the `voronoi_diagram` the number of colors has reduced to 3 (earlier it was 4).

We can implement other sorting algorithms, such as the DSATUR(Brelaz, 1979) which takes into account both the degree of the vertices and the degree of saturation of the vertices (DSAT) which signifies the number of colored vertices neighbouring a vertex. The DSATUR algorithm has 3 steps:
1. Choose the vertex with the maximum DSAT. If there are multiple equally high DSATs, choose the one with the minimal degree
2. Color this vertex with the smallest color possible
3. If all vertices are colored, stop. Else, repeat.

In [11]:
def DSATUR_greedy_coloring(adjlist):
    colors = [ None for i in range(len(adjlist))]
    while None in colors:
        DSAT = []
        maxDSAT = 0
        for i in range(len(adjlist)):
            if colors[i] is None:
                neighbors = adjlist[i]
                value = 0
                for neigh in neighbors:
                    if not colors[neigh] is None:
                        value += 1
                if value > maxDSAT:
                    DSAT = [i]
                    maxDSAT = value
                elif value == maxDSAT:
                    DSAT.append(i)
        maxDegree = 0
        valMaxDegree = 0
        for i in DSAT:
            if len(adjlist[i])>valMaxDegree:
                maxDegree = i
                valMaxDegree = len(adjlist[i])
                
        color = 0
        neighbors = adjlist[maxDegree]
        
        while True:
            isGood = True
            for i in neighbors:
                if not colors[i] is None:
                    if color == colors[i]:
                        isGood = False
                        break
                else:
                    pass
            if isGood:
                colors[maxDegree] = color
                break
            color += 1
    return colors

In [12]:
window = tk.Tk()
canvas = tk.Canvas(window, width=700, height=700)

draw_cartesian(canvas)

canvas.pack()
adj = voronoi_neighboring(voronoi_diagram_complete)
colors =DSATUR_greedy_coloring(adj)
draw_voronoi_colored(canvas, voronoi_diagram_complete, colors)
window.mainloop() 

![Results of DSATUR greedy coloring for the complete voronoi](images/colored_dsatur.jpg)
This algorithm succeds in coloring the `voronoi_diagram_complete` in just 4 colors!
NOTE: Do no run for `voronoi_diagram`. The algorithm requires that all germs have their cells defined.